<a href="https://colab.research.google.com/github/tuandung2812/Project-AI/blob/full/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install underthesea
! pip install torch-summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 26.4 MB/s 
     |████████████████████████████████| 235 kB 62.0 MB/s 
     |████████████████████████████████| 581 kB 65.3 MB/s 
     |████████████████████████████████| 965 kB 35.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import the necessary dependencies 

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import random
import numpy as np
import pandas as pd
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
from torch import nn
from tqdm.notebook import tqdm
from torch.autograd import Variable 
import matplotlib.pyplot as plt 
import torch.cuda
from sklearn.metrics import classification_report
import time
from google.colab import drive
from underthesea import word_tokenize
from string import punctuation

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True




## Run the following cells if you intend to run on Google Colab

In [ ]:
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/My\ Drive/Project-AI

/gdrive/My Drive/Project-AI


## Read our datasets

In [ ]:
train_df = pd.read_csv("./Data/processed/train.csv", sep = "|")
val_df = pd.read_csv("./Data/processed/validation.csv", sep = "|")
test_df = pd.read_csv("./Data/processed/test.csv", sep = "|")

In [ ]:
train_df

,sentences,sentiments
0,slide giáo_trình đầy_đủ,2
1,nhiệt_tình giảng_dạy gần_gũi với sinh_viên,2
2,đi học đầy_đủ full điểm chuyên cần,0
3,chưa áp_dụng công_nghệ_thông_tin và các thiết_...,0
4,thầy giảng bài hay có nhiều bài_tập ví_dụ ngay...,2
...,...,...
11421,chỉ vì môn game mà em học hai lần mà không qua...,0
11422,em cảm_ơn cô nhiều,2
11423,giao bài_tập quá nhiều,0
11424,giáo_viên dạy dễ hiểu nhiệt_tình,2


## Build vocabulary from the training set

In [ ]:
import collections
import os
import json
import datetime

class Vocabulary():
  '''Vocabulary class: extract all words from corpus, save all words that appears more than freq_threshold times'''
  def __init__(self, freq_threshold=3):
    self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
    self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
    self.freq_threshold = freq_threshold

  def __len__(self):
    return len(self.itos)

  def build_vocab(self, sentence_list):
    # Build the vocab
    idx = 4
    word_counter = collections.Counter([word for sentence in sentence_list for word in sentence])

    for word, count in word_counter.items():
      if count >= self.freq_threshold:
        self.stoi[word] = idx
        self.itos[idx] = word
        idx += 1

  def numericalize(self, sent):
    # Turn a sentence into list of word ID
    return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in sent
        ]
  def save_vocab(self, vocab_path):
    with open(vocab_path, 'w') as f:
        vocab = {'itos': self.itos, 'stoi': self.stoi}
        json.dump(vocab, f)

  def load_vocab(self, vocab_path):
    with open(vocab_path, 'r') as f:
        vocab = json.load(f)
        self.itos = vocab['itos']
        self.stoi = vocab['stoi']

In [ ]:
train_sents = train_df['sentences'].values
train_sents = [sent.split(" ") for sent in train_sents]
train_sents[0:3]

[['slide', 'giáo_trình', 'đầy_đủ'],
 ['nhiệt_tình', 'giảng_dạy', 'gần_gũi', 'với', 'sinh_viên'],
 ['đi', 'học', 'đầy_đủ', 'full', 'điểm', 'chuyên', 'cần']]

In [ ]:
vocab = Vocabulary(3)
vocab.build_vocab(train_sents)

In [ ]:
vocab.itos #string to index

{0: '<PAD>',
 1: '<SOS>',
 2: '<EOS>',
 3: '<UNK>',
 4: 'slide',
 5: 'giáo_trình',
 6: 'đầy_đủ',
 7: 'nhiệt_tình',
 8: 'giảng_dạy',
 9: 'gần_gũi',
 10: 'với',
 11: 'sinh_viên',
 12: 'đi',
 13: 'học',
 14: 'điểm',
 15: 'chuyên',
 16: 'cần',
 17: 'chưa',
 18: 'áp_dụng',
 19: 'công_nghệ_thông_tin',
 20: 'và',
 21: 'các',
 22: 'thiết_bị',
 23: 'hỗ_trợ',
 24: 'cho',
 25: 'việc',
 26: 'thầy',
 27: 'giảng',
 28: 'bài',
 29: 'hay',
 30: 'có',
 31: 'nhiều',
 32: 'bài_tập',
 33: 'ví_dụ',
 34: 'ngay',
 35: 'trên',
 36: 'lớp',
 37: 'giảng_viên',
 38: 'đảm_bảo',
 39: 'thời_gian',
 40: 'lên',
 41: 'tích_cực',
 42: 'trả_lời',
 43: 'câu_hỏi',
 44: 'của',
 45: 'thường_xuyên',
 46: 'đặt',
 47: 'câu',
 48: 'hỏi',
 49: 'em',
 50: 'sẽ',
 51: 'môn',
 52: 'này',
 53: 'nhưng',
 54: 'học_lại',
 55: 'ở',
 56: 'học_kỳ',
 57: 'thời_lượng',
 58: 'quá',
 59: 'dài',
 60: 'không',
 61: 'tiếp_thu',
 62: 'hiệu_quả',
 63: 'nội_dung',
 64: 'môn_học',
 65: 'phần',
 66: 'thiếu',
 67: 'trọng_tâm',
 68: 'hầu_như',
 69: 'là',

In [ ]:
vocab.stoi #string to index

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<UNK>': 3,
 'slide': 4,
 'giáo_trình': 5,
 'đầy_đủ': 6,
 'nhiệt_tình': 7,
 'giảng_dạy': 8,
 'gần_gũi': 9,
 'với': 10,
 'sinh_viên': 11,
 'đi': 12,
 'học': 13,
 'điểm': 14,
 'chuyên': 15,
 'cần': 16,
 'chưa': 17,
 'áp_dụng': 18,
 'công_nghệ_thông_tin': 19,
 'và': 20,
 'các': 21,
 'thiết_bị': 22,
 'hỗ_trợ': 23,
 'cho': 24,
 'việc': 25,
 'thầy': 26,
 'giảng': 27,
 'bài': 28,
 'hay': 29,
 'có': 30,
 'nhiều': 31,
 'bài_tập': 32,
 'ví_dụ': 33,
 'ngay': 34,
 'trên': 35,
 'lớp': 36,
 'giảng_viên': 37,
 'đảm_bảo': 38,
 'thời_gian': 39,
 'lên': 40,
 'tích_cực': 41,
 'trả_lời': 42,
 'câu_hỏi': 43,
 'của': 44,
 'thường_xuyên': 45,
 'đặt': 46,
 'câu': 47,
 'hỏi': 48,
 'em': 49,
 'sẽ': 50,
 'môn': 51,
 'này': 52,
 'nhưng': 53,
 'học_lại': 54,
 'ở': 55,
 'học_kỳ': 56,
 'thời_lượng': 57,
 'quá': 58,
 'dài': 59,
 'không': 60,
 'tiếp_thu': 61,
 'hiệu_quả': 62,
 'nội_dung': 63,
 'môn_học': 64,
 'phần': 65,
 'thiếu': 66,
 'trọng_tâm': 67,
 'hầu_như': 68,
 'là': 69,

## Declare PyTorch dataset and dataloader

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset
import random
import torch.nn.functional as F


seed  = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

class SentimentDataset(Dataset):
  def __init__(self,df, vocab = None, max_sent_len = 120):
    self.train_sents = [sent.split(" ") for sent in df['sentences'].values]
    self.labels = df['sentiments'].values
    self.max_sent_len = max_sent_len
    if vocab == None:
        self.vocab = Vocabulary(3)
        self.vocab.build_vocab(self.train_sents)
    else:
        self.vocab = vocab
  def __len__(self):
    return len(self.train_sents)
      
  def __getitem__(self, index):
    sent_numericalized = [self.vocab.stoi["<SOS>"]]
    sent_numericalized += self.vocab.numericalize(self.train_sents[index])
    
    if len(sent_numericalized) > self.max_sent_len:
        sent_numericalized = sent_numericalized[:self.max_sent_len]

    sent_tensor = np.full(self.max_sent_len, self.vocab.stoi["<PAD>"])
    sent_tensor[(self.max_sent_len - len(sent_numericalized)):] = sent_numericalized
    x = torch.Tensor(sent_tensor).long()
    
    y = self.labels[index]
    return x,y

In [ ]:
def get_loader(df, vocab = None, max_sent_len = 120,batch_size = 64, num_workers = 0, shuffle = True, pin_memory = True):
    dataset = SentimentDataset(df, vocab = vocab, max_sent_len = max_sent_len)
    vocab = dataset.vocab
    loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=shuffle,
        pin_memory=pin_memory
    )
    return loader, dataset, vocab

In [ ]:
train_loader,_,_ = get_loader(train_df, vocab = vocab, max_sent_len = 100)
val_loader,_,_ = get_loader(val_df, vocab = vocab, max_sent_len = 100)
test_loader,_,_ = get_loader(test_df, vocab = vocab, max_sent_len = 100)
batch_size = 64

## Declare our CNN model class

In [ ]:
class NetCNN(nn.Module):
    def __init__(self, vocab_size, embed_size, kernel_sizes, num_channels, drop, pad_idx):
        super(NetCNN, self).__init__()
        self.pad_idx = pad_idx
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx = pad_idx)
        
        # The embedding layer not to be trained
        self.constant_embedding = nn.Embedding(vocab_size, embed_size, padding_idx = pad_idx)
        self.dropout = nn.Dropout(drop)
        self.fc = nn.Linear(in_features = sum(num_channels), out_features = 3)

        # The max-over-time pooling layer has no parameters, so this instance
        # can be shared
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.relu = nn.ReLU()

        # Create multiple one-dimensional convolutional layers
        self.convs = nn.ModuleList()
        for c, k in zip(num_channels, kernel_sizes):
            self.convs.append(nn.Conv1d(2 * embed_size, c, k))

    def forward(self, inputs):
        # Concatenate two embedding layer outputs with shape (batch size, no.
        # of tokens, token vector dimension) along vectors
        embeddings = torch.cat((self.embedding(inputs), self.constant_embedding(inputs)), dim=2)

        # Per the input format of one-dimensional convolutional layers,
        # rearrange the tensor so that the second dimension stores channels
        embeddings = embeddings.permute(0, 2, 1)

        # For each one-dimensional convolutional layer, after max-over-time
        # pooling, a tensor of shape (batch size, no. of channels, 1) is
        # obtained. Remove the last dimension and concatenate along channels
        conv = torch.cat([
            torch.squeeze(self.relu(self.pool(conv(embeddings))), dim=-1)
            for conv in self.convs], dim=1)
        output = self.fc(self.dropout(conv))
        return output

## Create CNN model

In [ ]:
PAD_IDX = vocab.stoi["<PAD>"]
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 256
N_FILTERS = [100, 100, 100]
FILTER_SIZES = [3, 4, 5]
OUTPUT_DIM = 3
DROPOUT = 0.5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = NetCNN(INPUT_DIM, EMBEDDING_DIM, FILTER_SIZES, N_FILTERS, DROPOUT, PAD_IDX).to(device)

## Train CNN model

In [ ]:
# loss and optimization functions
lr=0.001
weight = torch.Tensor([1, 5.5, 1]).to(device)

criterion = nn.CrossEntropyLoss(weight = weight)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# training params

epochs = 10
counter = 0
print_every = 100 
clip = 5 # gradient clipping

print(device)

model.train()
# train for some number of epochs
for e in range(epochs):
    # batch loop
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        counter += 1
        #change device of labels and input to cuda, if available
        inputs = inputs.to(device)
        labels = labels.to(device)

        #set gradients of model to zero 
        model.zero_grad()

        #get the output from the input
        output = model(inputs)

        #calculate the loss and perform back-propagation
        loss = criterion(output, labels)
        loss.backward()

        #clip the gradient to prevent exploding gradient and using optimizer for the model
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        #evaluation in validation set
        if counter % print_every == 0:
            # Get validation loss
            val_losses = []
            model.eval()
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                #inputs = inputs.type(torch.LongTensor)
                output = model(inputs)

                val_loss = criterion(output, labels)

                val_losses.append(val_loss.item())

            model.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

cuda
Epoch: 1/10... Step: 100... Loss: 0.646307... Val Loss: 0.600607
Epoch: 2/10... Step: 200... Loss: 0.413452... Val Loss: 0.522596
Epoch: 2/10... Step: 300... Loss: 0.390427... Val Loss: 0.503364
Epoch: 3/10... Step: 400... Loss: 0.522911... Val Loss: 0.468547
Epoch: 3/10... Step: 500... Loss: 0.498127... Val Loss: 0.438261
Epoch: 4/10... Step: 600... Loss: 0.330040... Val Loss: 0.446327
Epoch: 4/10... Step: 700... Loss: 0.249068... Val Loss: 0.435114
Epoch: 5/10... Step: 800... Loss: 0.205459... Val Loss: 0.435396
Epoch: 6/10... Step: 900... Loss: 0.378734... Val Loss: 0.416370
Epoch: 6/10... Step: 1000... Loss: 0.195294... Val Loss: 0.425694
Epoch: 7/10... Step: 1100... Loss: 0.240081... Val Loss: 0.476554
Epoch: 7/10... Step: 1200... Loss: 0.275351... Val Loss: 0.422544
Epoch: 8/10... Step: 1300... Loss: 0.132173... Val Loss: 0.446670
Epoch: 8/10... Step: 1400... Loss: 0.220608... Val Loss: 0.459567
Epoch: 9/10... Step: 1500... Loss: 0.105217... Val Loss: 0.484782
Epoch: 9/10...

## Test CNN model

In [ ]:
#torch.save(model.state_dict(), "./model_CNN.pth.tar")
#model.load_state_dict(torch.load("./model_4.pth.tar"))
#initialization
test_losses = []
num_correct = 0

model.eval()
pred_total = torch.tensor([]).to(device)
labels_total = torch.tensor([]).to(device)

for inputs, labels in test_loader:

    #change device of labels and input to cuda, if available
    inputs = inputs.to(device)
    labels = labels.to(device)

    #get the output from the input
    output = model(inputs)
    
    # calculate loss
    test_loss = criterion(output, labels)
    test_losses.append(test_loss.item())

    #put predicted labels and true labels of each batch to the whole
    out = output.tolist()
    out = np.array(out)
    pred = torch.tensor(out.argmax(axis=1)).to(device)
    pred_total = torch.cat((pred_total, pred))
    labels_total = torch.cat((labels_total, labels))

pred_total = pred_total.tolist() 
labels_total = labels_total.tolist()

#report generating
print("Test loss: {:.3f}".format(np.mean(test_losses)))
target_names = ['negative', 'neutral', 'positive']
print(classification_report(labels_total, pred_total, target_names=target_names, digits = 4))

Test loss: 0.502
              precision    recall  f1-score   support

    negative     0.9227    0.9149    0.9188       705
     neutral     0.4393    0.6438    0.5222        73
    positive     0.9537    0.9205    0.9368       805

    accuracy                         0.9052      1583
   macro avg     0.7719    0.8264    0.7926      1583
weighted avg     0.9162    0.9052    0.9097      1583



## Get the classification results from model

In [ ]:
def get_predict(reviews):
  pred = []
  res = []
  set_encoded_review = []

  #process the input
  for review in reviews:
    review = review.lower()
    review = [review[i] for i in range(len(review)) if review[i] not in punctuation]
    review = "".join(map(str, review))
    splited_review = word_tokenize(review, format = "text").split()
    encoded_review = vocab.numericalize(splited_review)
    temp = np.full(100, vocab.stoi["<PAD>"])
    temp[(100 - len(encoded_review)):] = encoded_review
    set_encoded_review.append(temp.tolist())
  input = torch.tensor(set_encoded_review).to(device)

  #get the output from the model
  output = model(input)

  #process the output to desired results
  pred_prob = np.squeeze(F.softmax(output, dim = 1)).tolist()
  pred = np.squeeze(np.array(output.tolist()).argmax(axis = 1)).tolist()

  if type(pred) == int :
    pred = [pred]

  for mem in pred:
    if mem == 0:
      res.append("tiêu cực")
    elif mem == 1:
      res.append("trung lập")
    else:
      res.append("tích cực")

  return res, pred_prob

##Test some real-life feedbacks

In [ ]:
reviews = ["các yêu cầu về đồ án môn học nên đươc đưa ra chính xác trước khi sinh viên bắt đầu thực hiện chia nhóm và làm đồ án", 
           "dạy dễ hiểu dễ tiếp thu tập trung vài kiến thức trọng tâm", 
           "cần cải cách lại nội dung các bài thực hành một số kiến nội dung trong bài thực hành đã trở nên lỗi thời", 
           "thầy dạy hơi nhanh", 
           "chúng ta không thuộc về nhau", 
           "đề tài em chọn trong bài tập lớn là phân phối chuẩn",
           "vshjhuajvjfhùbdjsfhdkhg98765464fjhghjkgjfjhkhngf"]
res = get_predict(reviews)
print(res[0])
print(res[1])

['tiêu cực', 'tích cực', 'tiêu cực', 'tiêu cực', 'trung lập', 'tích cực', 'trung lập']
[[0.9846903681755066, 0.007602235302329063, 0.007707320153713226], [0.00021355813078116626, 4.3454753040350624e-07, 0.9997859597206116], [0.998969316482544, 3.283066689618863e-05, 0.0009978004964068532], [0.5919790863990784, 0.3523547947406769, 0.05566607788205147], [0.4146498143672943, 0.5780010223388672, 0.007349169347435236], [0.0499969907104969, 0.17056162655353546, 0.7794414162635803], [0.2552280128002167, 0.5302879214286804, 0.2144840657711029]]


## Console interface

In [ ]:
#Console interface

usr_review = input("Hãy nhập nhận xét của bạn vào đây, nhấn Enter để xác nhận: ")
usr_review = [usr_review]
sent, prob = get_predict(usr_review)
while (True):
  print("Bạn muốn đưa ra kết quả dưới dạng nào? ")
  print("1. Đưa ra dự đoán về thuộc tính của nhận xét")
  print("2. Đưa ra xác suất nhận xét có một trong các thuộc tính")
  choice = int(input("Hãy nhập lựa chọn của bạn [1-2]: "))
  if choice == 1:
    print("Nhận xét của bạn mang tính", sent[0])
    break
  elif choice == 2:
    neg = prob[0]*100
    neu = prob[1]*100
    pos = prob[2]*100
    print("Xác suất nhận xét của bạn mang tính tiêu cực: {:.2f}%".format(neg))
    print("Xác suất nhận xét của bạn mang tính trung lập: {:.2f}%".format(neu))
    print("Xác suất nhận xét của bạn mang tính tích cực: {:.2f}%".format(pos))
    break
  else:
    print("Lựa chọn không hợp lệ. Xin hãy nhập lại lựa chọn theo như hướng dẫn.")

Hãy nhập nhận xét của bạn vào đây, nhấn Enter để xác nhận: Thật thú vị
Bạn muốn đưa ra kết quả dưới dạng nào? 
1. Đưa ra dự đoán về thuộc tính của nhận xét
2. Đưa ra xác suất nhận xét có một trong các thuộc tính
Hãy nhập lựa chọn của bạn [1-2]: 1
Nhận xét của bạn mang tính tích cực
